In [19]:
import csv
import json
import datetime
import uuid
import pandas as pd
import requests
import time
import openai
import os
from dotenv import load_dotenv
import hashlib
import concurrent.futures
import time

In [20]:
CAMINHO_BASE = '../'
DIRETORIO_DADOS = CAMINHO_BASE + 'data/'

In [21]:
load_dotenv() # Busca .env (normalmente na raiz do projeto) e carrega as variáveis de ambiente
openai.api_key = os.getenv("OPENAI_API_KEY", "sk-xxxxx") # Define a chave da API do OpenAI com a variável de ambiente OPENAI_API_KEY, se não existir, usa a chave padrão (sk-xxxxx) e não vai funcionar.
CANDIDATO = os.getenv("CANDIDATO", "xxxxx") 
CARGO = os.getenv("CARGO", "xxxxx") 
LOCALIDADE = os.getenv("LOCALIDADE", "xxxxx") 
É_REELEIÇÃO = os.getenv("É_REELEIÇÃO", "xxxxx") 
CAMPANHA_ORIGINAL = os.getenv("CAMPANHA_ORIGINAL", "xxxxx") 

In [22]:
def hash_texto(texto) -> str:
    return hashlib.md5(texto.encode()).hexdigest()[:8]

In [23]:
def assegura_diretorio_campanha(diretorio):
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)

In [24]:
def calcular_margem_tamanho_texto(texto, margem = 0.1, saida_minima = False, saida_maxima = False) -> int:
    margem = int(len(texto) * margem)
    if saida_minima:
        return len(texto) - margem
    if saida_maxima:
        return len(texto) + margem
    return margem

In [25]:
def montar_detalhe_reeleicao(é_reeleição) -> str:
    if é_reeleição:
        return 'e é o atual gestor em busca da reeleição.'
    return ', entretanto não é o atual gestor'

In [26]:
def reescreveTexto(texto_campanha, prompt_sistema, temperatura = 0.8) -> str:
    try:
        # Usa o novo método `ChatCompletion.create` para interagir com o modelo
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # O modelo "gpt-4-turbo" é o mais recente e suportado
            messages=[
                {"role": "system", "content": prompt_sistema},
                {"role": "user", "content": texto_campanha}
            ],
            max_tokens=2000,
            temperature=temperatura,
        )
        
        # Obtém o texto reescrito
        return response['choices'][0]['message']['content'].strip()
    
    except Exception as e:
        print("Erro ao se comunicar com a API do OpenAI:", e)
        raise e

In [27]:
def salva_campanhas_alternativas_json(diretorio_campanha, lista_campanhas_alternativas):
    caminho_arquivo = diretorio_campanha + 'campanhas_alternativas.json'
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        json.dump(lista_campanhas_alternativas, f, ensure_ascii=False, indent=4)

In [28]:
def reescreve_e_salva_campanha(diretorio_dados, campanha_original, prompt_sistema, versões_alternativas = 10, temperatura = 0.8) -> list:
    campanhas_alternativas = []
    campanhas_alternativas.append(campanha_original)
    
    def gerar_campanha_alternativa(index):
        campanha_alternativa = reescreveTexto(campanha_original, prompt_sistema, temperatura)
        print(f'Gerando {index + 1}º versão de campanha alternativa...')
        return campanha_alternativa
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(gerar_campanha_alternativa, i) for i in range(versões_alternativas)]
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            print(f'{completed}/{versões_alternativas}')
            campanhas_alternativas.append(future.result())
            time.sleep(1)  # Se precisar manter o intervalo de tempo entre as execuções
    
    hash_campanha = hash_texto(campanha_original)
    diretorio_campanha = diretorio_dados + hash_campanha + '/'
    assegura_diretorio_campanha(diretorio_campanha)
    salva_campanhas_alternativas_json(diretorio_campanha, campanhas_alternativas)
    print(f'Campanhas alternativas salvas em {diretorio_campanha}')
    return campanhas_alternativas

In [31]:
PROMPT_SISTEMA = f"Você é um assistente virtual que está ajudando {CANDIDATO}, candidato a {CARGO} da cidade de {LOCALIDADE} a reescrever textos para sua campanha eleitoral. {CANDIDATO} é candidato a {CARGO} {montar_detalhe_reeleicao(É_REELEIÇÃO)}. Lhe será fornecido um texto e você deverá reescrevê-lo de forma a torná-lo ligeiramente diferente, mas mantendo o mesmo sentido e a mesma mensagem. Você não deve adicionar novas informações ao texto ou imaginar situações que não estão presentes no texto original. Troque palavras por sinônimos, reorganize frases, altere a ordem de parágrafos, altere conectores ou outras alterações que julgar necessárias. O texto reescrito deve possuir de {calcular_margem_tamanho_texto(CAMPANHA_ORIGINAL, saida_minima=True)} a {calcular_margem_tamanho_texto(CAMPANHA_ORIGINAL, saida_maxima=True)} caracteres. Mantenha as hashtags presentes no texto original."

In [32]:
campanhas_alternativas = reescreve_e_salva_campanha(
    DIRETORIO_DADOS,
    CAMPANHA_ORIGINAL,
    PROMPT_SISTEMA, 
    versões_alternativas=20,
    temperatura=0.75
    ) 

Gerando 18º versão de campanha alternativa...
1/20
Gerando 6º versão de campanha alternativa...
Gerando 8º versão de campanha alternativa...
Gerando 14º versão de campanha alternativa...
Gerando 2º versão de campanha alternativa...
Gerando 16º versão de campanha alternativa...
Gerando 5º versão de campanha alternativa...
Gerando 13º versão de campanha alternativa...
Gerando 12º versão de campanha alternativa...
Gerando 9º versão de campanha alternativa...
Gerando 4º versão de campanha alternativa...
Gerando 15º versão de campanha alternativa...
Gerando 1º versão de campanha alternativa...
Gerando 17º versão de campanha alternativa...
Gerando 11º versão de campanha alternativa...
Gerando 19º versão de campanha alternativa...
Gerando 10º versão de campanha alternativa...
Gerando 7º versão de campanha alternativa...
2/20
Gerando 20º versão de campanha alternativa...
Gerando 3º versão de campanha alternativa...
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17